In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings("ignore")
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
test_ID = test['ID']
y_train = train['target']
y_train = np.log1p(y_train)
train.drop("ID", axis = 1, inplace = True)
train.drop("target", axis = 1, inplace = True)
test.drop("ID", axis = 1, inplace = True)
#train.shape
#(4459, 4992)
#test.shape
#(49342, 4991)
cols_with_onlyone_val = train.columns[train.nunique() == 1]
train.drop(cols_with_onlyone_val.values, axis=1, inplace=True)
test.drop(cols_with_onlyone_val.values, axis=1, inplace=True)
#(4459, 4735)
#train.drop_duplicates(inplace=True)
#test.drop_duplicates(inplace=True)
#train.shape
train = train.round(32)
test = test.round(32)

colsToRemove = []

columns = train.columns

for i in range(len(columns)-1):
    v = train[columns[i]].values
    dupCols = []
    for j in range(i + 1,len(columns)):
        if np.array_equal(v, train[columns[j]].values):
            colsToRemove.append(columns[j])
train.drop(colsToRemove, axis=1, inplace=True) 
test.drop(colsToRemove, axis=1, inplace=True) 
colsToRemove
#['d60ddde1b', 'acc5b709d', 'f333a5f60', 'f8d75792f', '912836770', 'f333a5f60']

['.DS_Store', 'test.csv', 'train.csv', 'test.csv.zip', 'train.csv.zip', 'sample_submission.csv.zip', 'sample_submission.csv']


['d60ddde1b', 'acc5b709d', 'f333a5f60', 'f8d75792f', '912836770', 'f333a5f60']

In [2]:
#add additional features
from sklearn import random_projection
from sklearn.preprocessing import scale, MinMaxScaler
import gc
import itertools
from copy import deepcopy
import time
import progressbar

ntrain = len(train)
ntest = len(test)
weight = ((train != 0).sum()/len(train)).values

tmp_train = train[train!=0]
tmp_test = test[test!=0]
tmp = pd.concat([train,test])#RandomProjection
train["weight_count"] = (tmp_train*weight).sum(axis=1)
test["weight_count"] = (tmp_test*weight).sum(axis=1)
train["count_not0"] = (train != 0).sum(axis=1)
test["count_not0"] = (test != 0).sum(axis=1)
train["sum"] = train.sum(axis=1)
test["sum"] = test.sum(axis=1)
train["var"] = tmp_train.var(axis=1)
test["var"] = tmp_test.var(axis=1)
train["mean"] = tmp_train.mean(axis=1)
test["mean"] = tmp_test.mean(axis=1)
train["std"] = tmp_train.std(axis=1)
test["std"] = tmp_test.std(axis=1)
train["max"] = tmp_train.max(axis=1)
test["max"] = tmp_test.max(axis=1)
train["min"] = tmp_train.min(axis=1)
test["min"] = tmp_test.min(axis=1)
del(tmp_train)
del(tmp_test)

# train data is valid , test data has nan and infinite
tmp = pd.DataFrame(np.nan_to_num(tmp))
# Go through the columns one at a time (can't do it all at once for this dataset)
total_df = deepcopy(tmp)      
print('np.any(np.isnan(total_df)', np.any(np.isnan(total_df)))
print('np.all(np.isfinite(total_df)', np.all(np.isfinite(total_df)))

p = progressbar.ProgressBar()
p.start()

# Mean-variance scale all columns excluding 0-values'
print('total_df.columns:',total_df.columns) 
columnsCount = len(total_df.columns)
for col in total_df.columns:    
    p.update(col/columnsCount * 100)

    # Detect outliers in this column
    data = total_df[col].values
    data_mean, data_std = np.mean(data), np.std(data)
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers = [x for x in data if x < lower or x > upper]
    
    # If there are crazy high values, do a log-transform
    if len(outliers) > 0:
        non_zero_idx = data != 0
        total_df.loc[non_zero_idx, col] = np.log(data[non_zero_idx])
    
    # Scale non-zero column values
    nonzero_rows = total_df[col] != 0
    if  np.isfinite(total_df.loc[nonzero_rows, col]).all():
        total_df.loc[nonzero_rows, col] = scale(total_df.loc[nonzero_rows, col])
        if  np.isfinite(total_df[col]).all():
            # Scale all column values
            total_df[col] = scale(total_df[col])
    gc.collect()
    
p.finish()

n_com = 100
transformer = random_projection.SparseRandomProjection(n_components = n_com)
RP = transformer.fit_transform(total_df)
rp = pd.DataFrame(RP)
columns = ["RandomProjection{}".format(i) for i in range(n_com)]
rp.columns = columns

rp_train = rp[:ntrain]
rp_test = rp[ntrain:]
rp_test.index = test.index

#concat RandomProjection and raw data
train = pd.concat([train,rp_train],axis=1)
test = pd.concat([test,rp_test],axis=1)

del(rp_train)
del(rp_test)

np.any(np.isnan(total_df) False


np.all(np.isfinite(total_df) True
total_df.columns: RangeIndex(start=0, stop=4730, step=1)


100% |########################################################################|


In [3]:
#modeling
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
#define evaluation method for a given model. we use k-fold cross validation on the training set. 
#the loss function is root mean square logarithm error between target and prediction
#note: train and y_train are feeded as global variables
n_folds = 5
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)
    
#base models with hyper parameters already tuned
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=1, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
#model_xgb = xgb.XGBRegressor(colsample_bytree=0.5, eval_metric = 'rmse', objective = 'reg:linear',
#                             eta=0.005, subsample=0.7, alpha=0, silent = True,
#                             learning_rate=0.02, max_depth=10)
#params = {'objective': 'reg:linear', 'eval_metric': 'rmse', 'eta': 0.005, 'max_depth': 10, 
#          'subsample': 0.7, 'colsample_bytree': 0.5, 'alpha':0, 'silent': True, 'random_state':5}
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=144,
                              learning_rate=0.005, n_estimators=720, max_depth=13,
                              metric='rmse',is_training_metric=True,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.9)

model_cb = CatBoostRegressor(iterations=500,learning_rate=0.05,depth=10,
                           eval_metric='RMSE',bagging_temperature = 0.2,
                           od_type='Iter',metric_period = 50,od_wait=20)
#ensemble method 1: model averaging
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    # the reason of clone is avoiding affect the original base models
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]  
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)
        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([ model.predict(X) for model in self.models_ ])
        return np.mean(predictions, axis=1)  

#ensemble method 2: staking models
class StackedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [4]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
score = rmsle_cv(model_cb)
print("catBoost score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
averaged_models = AveragingModels(models = (model_xgb, model_lgb, model_cb))
score = rmsle_cv(averaged_models)
print("averaged_models score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
stacked_models = StackedModels(base_models = (ENet, model_xgb, GBoost), meta_model = lasso)
score = rmsle_cv(stacked_models)
print("Stacked models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

#for 100 features, round 32
#Lasso score: 1.6861 (0.0485)
#ElasticNet score: 1.6861 (0.0485)
#Kernel Ridge score: 357872.8763 (72971.0130)
#Gradient Boosting score: 1.4765 (0.0460)
#Xgboost score: 1.4441 (0.0483)
#LGBM score: 1.4366 (0.0445)
#averaged_models score: 1.4293 (0.0473)

#for 100 features, round 16
#Lasso score: 1.6861 (0.0485)
#ElasticNet score: 1.6861 (0.0485)
#Kernel Ridge score: 357698.7189 (72988.5463)
#Gradient Boosting score: 1.4757 (0.0418)
#Xgboost score: 1.4457 (0.0465)
#LGBM score: 1.4371 (0.0442)
#averaged_models score: 1.4291 (0.0459)

#for all features + aggragate features, average between xgb and lgb
#Xgboost score: 1.3779 (0.0501)
#LGBM score: 1.3555 (0.0477)
#averaged_models score: 1.3574 (0.0492)

#for rounded to 16, 400 by importance, drop disimilar test/train features, add aggragate features
#Xgboost score: 2.3424 (0.0390)
#LGBM score: 1.3543 (0.0551)
#averaged_models score: 1.6546 (0.0414)

#for rounded to 16, 200 by importance, drop disimilar test/train features, add aggragate features
#Xgboost score: 1.4161 (0.0440)
#LGBM score: 1.3653 (0.0476)
#averaged_models score: 1.3747 (0.0466)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
#submit 
model_lgb.fit(train.values, y_train)
pred = np.expm1(model_lgb.predict(test.values))
ensemble = pred
sub = pd.DataFrame()
sub['ID'] = test_ID
sub['target'] = ensemble
sub.to_csv('submission.csv',index=False)